# Training a NN to Improve Raw ORB/RANSAC Outputs

In [1]:
# Read in data from nn_X.csv and nn_Y.csv. Train a neural network on the data.

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Read in data. Ignore the chain_id column and the i column.
X = pd.read_csv('nn_X.csv').drop(columns=['chain_id', 'i']).to_numpy()
Y = pd.read_csv('nn_Y.csv').drop(columns=['chain_id', 'i']).to_numpy()


# DATA NORMALISATION
# Normalise the range column from [20, 600] to [0, 1].
X[:, 0] = (X[:, 0] - 20) / (600 - 20)

# Normalise the x, y, z columns from [-1000, 1000] to [-1, 1].
X[:, 1:4] = (X[:, 1:4] - (-1000)) / (1000 - (-1000))
Y[:, 0:3] = (Y[:, 0:3] - (-1000)) / (1000 - (-1000))

# FEATURE ENGINEERING
# For each of the columns x, y, z, qw, qx, qy, qz, add a column x^2, y^2, z^2, etc.
# This is to help it learn formulae that depend on the square of these values.
X = np.hstack([X, X[:, 3:]**2])

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Convert data to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

# Create neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(Y.shape[1])
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train neural network
model.fit(X_train, Y_train, epochs=500, batch_size=25, validation_data=(X_test, Y_test), verbose=2)

# Evaluate neural network
Y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean squared error: {mse}')


2024-04-19 16:43:11.658647: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/500


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 2s - 52ms/step - loss: 0.1474 - val_loss: 0.0271
Epoch 2/500
32/32 - 0s - 4ms/step - loss: 0.0438 - val_loss: 0.0191
Epoch 3/500
32/32 - 0s - 4ms/step - loss: 0.0369 - val_loss: 0.0177
Epoch 4/500
32/32 - 0s - 4ms/step - loss: 0.0338 - val_loss: 0.0179
Epoch 5/500
32/32 - 0s - 4ms/step - loss: 0.0329 - val_loss: 0.0165
Epoch 6/500
32/32 - 0s - 4ms/step - loss: 0.0306 - val_loss: 0.0166
Epoch 7/500
32/32 - 0s - 4ms/step - loss: 0.0284 - val_loss: 0.0165
Epoch 8/500
32/32 - 0s - 4ms/step - loss: 0.0274 - val_loss: 0.0164
Epoch 9/500
32/32 - 0s - 4ms/step - loss: 0.0274 - val_loss: 0.0163
Epoch 10/500
32/32 - 0s - 4ms/step - loss: 0.0260 - val_loss: 0.0162
Epoch 11/500
32/32 - 0s - 6ms/step - loss: 0.0253 - val_loss: 0.0208
Epoch 12/500
32/32 - 0s - 6ms/step - loss: 0.0263 - val_loss: 0.0162
Epoch 13/500
32/32 - 0s - 4ms/step - loss: 0.0250 - val_loss: 0.0165
Epoch 14/500
32/32 - 0s - 4ms/step - loss: 0.0244 - val_loss: 0.0163
Epoch 15/500
32/32 - 0s - 4ms/step - loss: 0.0253 - va

# Saving the Model

In [2]:
# Save the model.
model.save('my_model.keras')